In [ ]:
!pip install datasets

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, TrainingArguments, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset, Dataset
import torch
import json
import re

In [ ]:
with open("symptoms_diagnoses_100.json", "r", encoding="utf-8") as f:
    data = json.load(f)

def format_example(example):
    return {
        "text": f"علائم بیمار: {example['symptom']}\nتشخیص: {example['diagnosis']}"
    }

dataset = Dataset.from_list(data)
dataset = dataset.map(format_example)


In [ ]:
print(dataset)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("unsloth/Llama-3.2-1B")

In [ ]:
def evaluate_model(model, tokenizer):
    correct = 0
    results = []

    with open("qa.json", "r", encoding="utf-8") as f:
        questions = json.load(f)

    for q in questions:
        prompt = f"""در ادامه یک سؤال چهارگزینه‌ای پزشکی آمده است. فقط یکی از گزینه‌ها صحیح است. لطفاً فقط شماره گزینه صحیح را بنویس.\n\nسؤال: {q["question"]}\n"""
        for i, opt in enumerate(q["options"], 1):
            prompt += f"{i}. {opt}\n"
        prompt += "\nپاسخ:"

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=20,
                do_sample=False
            )

        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predicted_text = decoded.split("پاسخ:")[-1].strip()
        predicted_answer = predicted_text.split("\n")[0]

        try:
            is_correct = int(re.sub(r"\D", "", predicted_answer)) == q["answer"]
        except:
            is_correct = False

        if is_correct:
            correct += 1

        results.append({
            "question": q["question"],
            "predicted": predicted_answer,
            "correct": q["answer"],
            "is_correct": is_correct
        })

    accuracy = correct / len(questions)
    return accuracy, results


In [ ]:
acc_before, results_before = evaluate_model(model, tokenizer)
print(f"دقت مدل پایه: {acc_before:.2%}")

with open("evaluation_base_model.json", "w", encoding="utf-8") as f:
    json.dump(results_before, f, ensure_ascii=False, indent=2)

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

In [ ]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f" تعداد پارامترهای قابل آموزش: {trainable_params:,}")
print(f"درصد از کل مدل: {100 * trainable_params / total_params:.2f}%")

In [ ]:
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=384)

tokenized_dataset = dataset.map(tokenize, batched=True)


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=4e-5,
    logging_dir="./logs",
    save_steps=500,
    logging_steps=100,
    fp16=False,
    save_total_limit=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


trainer.train()

trainer.save_model("./fine-tuned-model")
tokenizer.save_pretrained("./fine-tuned-model")

In [ ]:
acc_after, results_after = evaluate_model(model, tokenizer)
print(f"دقت مدل fine-tuned: {acc_after:.2%}")

with open("evaluation_finetuned_model.json", "w", encoding="utf-8") as f:
    json.dump(results_after, f, ensure_ascii=False, indent=2)